Test Run for mass testing

### Get company info 

In [1]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

In [2]:
start = datetime.datetime(2020,2,1)
end = datetime.datetime(2020,10,11)


In [3]:
url="https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv"
s = requests.get(url).content
companies = pd.read_csv(io.StringIO(s.decode('utf-8')))
Symbols = companies['Symbol'].tolist()
#Symbols[:10]

In [4]:
## Get short volume data 

In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 5]

import numpy as np
import fnmatch
import os
import pandas as pd
import datetime

TICKERS = Symbols

In [6]:
def get_df(sdir, NAME, raw_x, TICKER ):
    '''
    This function iterates through RegSHO data over a data range and pulls rows for a TICKER and
    puts the data into a pandas data frame
    a row is as follows. The consolodated daily short data is in the CNMSyyyymmdd.txt file for every trade day
    
    Date|Symbol|ShortVolume|ShortExemptVolume|TotalVolume|Market
    
    '''
    xxx = []
    for shortdate in raw_x:
        #print(shortdate)
        for file in os.listdir(sdir):
            matchme = prefix+str(shortdate)+'*'
            if fnmatch.fnmatch(file, matchme):
                xxx.append(file)

    df = None            


    for filename in xxx:
        f = open(sdir+filename)
        try:
            fullfile = pd.read_csv(f,sep="|")
            if df is None:
                tickerrow = fullfile.loc[fullfile['Symbol'] == TICKER]
                df = tickerrow

            else:
                tickerrow = fullfile.loc[fullfile['Symbol'] == TICKER]
                df = df.append(tickerrow)
        except:
            print('failed: {}'.format(filename))
    return df

## Load up the data

We have to build the filename it follows the format

```CNSMshvolYYYYMMDD.txt```

In [7]:
def get_df(sdir, prefix, raw_x, TICKER ):
    '''
    This function iterates through RegSHO data over a data range and pulls rows for a TICKER and
    puts the data into a pandas data frame
    a row is as follows. The consolodated daily short data is in the CNMSyyyymmdd.txt file for every trade day
    
    Date|Symbol|ShortVolume|ShortExemptVolume|TotalVolume|Market
    
    '''
    xxx = []
    for shortdate in raw_x:
        #print(shortdate)
        for file in os.listdir(sdir):
            matchme = prefix+str(shortdate)+'*'
            if fnmatch.fnmatch(file, matchme):
                xxx.append(file)

    df = None            


    for filename in xxx:
        f = open(sdir+filename)
        try:
            fullfile = pd.read_csv(f,sep="|")
            if df is None:
                tickerrow = fullfile.loc[fullfile['Symbol'] == TICKER]
                df = tickerrow

            else:
                tickerrow = fullfile.loc[fullfile['Symbol'] == TICKER]
                df = df.append(tickerrow)
        except:
            pass
            #print('failed: {}'.format(filename))
    print('completed loading')
    return df

In [8]:
def getShortVolumeData(TICKER, startdate, enddate):
    x = pd.date_range(start=startdate,end=enddate,freq='D').strftime('%Y%m%d')

    # this is the location of where I downloaded the data. REPLACE this with where you have your data 
    sdir =  '/home/pookie/data/shortdata2/'

    # Base Filename Generation
    prefix = 'CNMSshvol'
    #prefix = 'FNSQshvol'
    # Our test Ticker.. 
    #TICKER = 'AMD'

    df = get_df(sdir, prefix, x, TICKER)
    return df

In [9]:
def makeDataFrame(SYMBOL, df, mktcap,
                 startdate, enddate):
    # get stock daily pricing
    #TICKER = 'GME'
    #derp = dataframe_collection[TICKER]
    #mktcap = shares_collection[TICKER]


    start = datetime.datetime.strptime(startdate,'%Y%m%d')
    end = datetime.datetime.strptime(enddate,'%Y%m%d')

    stock = yf.download(SYMBOL,start=start.strftime('%Y-%m-%d'), end=end.strftime('%Y-%m-%d'), progress=False)

    format = '%Y%m%d'
    df['Datetime'] = pd.to_datetime(df['Date'] , format=format)
    df = df.set_index(pd.DatetimeIndex(df['Datetime']))

    del df['Date']
    del df['Datetime']

    df = df.merge(stock, left_index=True, right_index=True, how='inner')
    df['NetShort'] = df.TotalVolume - 2*(df.ShortVolume+df.ShortExemptVolume)
    df['NSValueClose'] = df.NetShort * df.Close 
    df['NSValueOpen'] = df.NetShort * df.Open 
    df['NetShortPercent'] = df.NetShort / mktcap *100
    df['CumNetShort'] =   np.cumsum(df.NetShort)
    df['CumNetShortPercent'] =   df.CumNetShort / mktcap *100
    return df
    

In [10]:
startdate = '20180101'
enddate = '20220101'
initial_short_position = 0



In [ ]:
#TICKERS = ['AAPL','GME', 'F', 'BBY', 'KOSS', 'XLNX', 'AMD', 'TSLA', 'EVGO']

shares_collection = {}
dataframe_collection = {}
for SYMBOL in TICKERS: 
    try: 
        TICK = yf.Ticker(SYMBOL)
        mktcap = TICK.info['sharesOutstanding']
        df_shortvol = getShortVolumeData(SYMBOL, startdate, enddate)
        dataframe_collection[SYMBOL] = makeDataFrame(SYMBOL, df_shortvol, mktcap,  startdate, enddate)
        shares_collection[SYMBOL] = TICK.info['sharesOutstanding']
    except:
        print('Errror with {}, {}'.format(SYMBOL, TICK))


Errror with AAIT, yfinance.Ticker object <AAIT>
completed loading
completed loading
completed loading
completed loading
completed loading
Errror with AAVL, yfinance.Ticker object <AAVL>
completed loading
Errror with AAXJ, yfinance.Ticker object <AAXJ>
Errror with ABAC, yfinance.Ticker object <ABAC>
Errror with ABAX, yfinance.Ticker object <ABAX>
completed loading
Errror with ABCD, yfinance.Ticker object <ABCD>
Errror with ABCO, yfinance.Ticker object <ABCO>
Errror with ABCW, yfinance.Ticker object <ABCW>
Errror with ABDC, yfinance.Ticker object <ABDC>
Errror with ABGB, yfinance.Ticker object <ABGB>
completed loading
completed loading
Errror with ABTL, yfinance.Ticker object <ABTL>
Errror with ABY, yfinance.Ticker object <ABY>
completed loading
Errror with ACAS, yfinance.Ticker object <ACAS>
Errror with ACAT, yfinance.Ticker object <ACAT>
completed loading
Errror with ACFC, yfinance.Ticker object <ACFC>
completed loading
completed loading
completed loading
Errror with ACHN, yfinance.Tic

completed loading
completed loading
Errror with AVHI, yfinance.Ticker object <AVHI>
completed loading
Errror with AVNR, yfinance.Ticker object <AVNR>
completed loading
Errror with AWAY, yfinance.Ticker object <AWAY>
completed loading
completed loading
completed loading
completed loading
Errror with AXJS, yfinance.Ticker object <AXJS>
Errror with AXPW, yfinance.Ticker object <AXPW>
Errror with AXPWW, yfinance.Ticker object <AXPWW>
completed loading
completed loading
Errror with BABY, yfinance.Ticker object <BABY>
Errror with BAGR, yfinance.Ticker object <BAGR>
Errror with BAMM, yfinance.Ticker object <BAMM>
completed loading
Errror with BANFP, yfinance.Ticker object <BANFP>
completed loading
completed loading
Errror with BASI, yfinance.Ticker object <BASI>
completed loading
Errror with BBC, yfinance.Ticker object <BBC>
Errror with BBCN, yfinance.Ticker object <BBCN>
Errror with BBEP, yfinance.Ticker object <BBEP>
Errror with BBEPP, yfinance.Ticker object <BBEPP>
completed loading
Errror

completed loading
Errror with CFGE, yfinance.Ticker object <CFGE>
completed loading
Errror with CFNL, yfinance.Ticker object <CFNL>
Errror with CFO, yfinance.Ticker object <CFO>
completed loading
Errror with CFRXW, yfinance.Ticker object <CFRXW>
Errror with CFRXZ, yfinance.Ticker object <CFRXZ>
completed loading
completed loading
Errror with CGIX, yfinance.Ticker object <CGIX>
completed loading
completed loading
completed loading
completed loading
completed loading
completed loading
Errror with CHEV, yfinance.Ticker object <CHEV>
Errror with CHFC, yfinance.Ticker object <CHFC>
Errror with CHFN, yfinance.Ticker object <CHFN>
completed loading
Errror with CHKE, yfinance.Ticker object <CHKE>
completed loading
Errror with CHLN, yfinance.Ticker object <CHLN>
completed loading
completed loading
Errror with CHOP, yfinance.Ticker object <CHOP>
completed loading
completed loading
completed loading
Errror with CHSCM, yfinance.Ticker object <CHSCM>
completed loading
Errror with CHSCN, yfinance.Ti

Errror with DGAS, yfinance.Ticker object <DGAS>
completed loading
completed loading
completed loading
Errror with DGLD, yfinance.Ticker object <DGLD>
completed loading
Errror with DGRE, yfinance.Ticker object <DGRE>
Errror with DGRS, yfinance.Ticker object <DGRS>
Errror with DGRW, yfinance.Ticker object <DGRW>
completed loading
Errror with DHRM, yfinance.Ticker object <DHRM>
completed loading
completed loading
completed loading
completed loading
completed loading
completed loading
Errror with DLBL, yfinance.Ticker object <DLBL>
Errror with DLBS, yfinance.Ticker object <DLBS>
completed loading
completed loading
completed loading
Errror with DMND, yfinance.Ticker object <DMND>
completed loading
Errror with DNBF, yfinance.Ticker object <DNBF>
Errror with DNKN, yfinance.Ticker object <DNKN>
completed loading
Errror with DOVR, yfinance.Ticker object <DOVR>
completed loading
Errror with DPRX, yfinance.Ticker object <DPRX>
Errror with DRAD, yfinance.Ticker object <DRAD>
Errror with DRAM, yfin

completed loading
completed loading
completed loading
Errror with FHCO, yfinance.Ticker object <FHCO>
completed loading
Errror with FINL, yfinance.Ticker object <FINL>
Errror with FISH, yfinance.Ticker object <FISH>
completed loading
completed loading
completed loading
completed loading
completed loading
completed loading
completed loading
Errror with FLAT, yfinance.Ticker object <FLAT>
completed loading
completed loading
completed loading
Errror with FLIR, yfinance.Ticker object <FLIR>
completed loading
Errror with FLML, yfinance.Ticker object <FLML>
completed loading
completed loading
completed loading
Errror with FMB, yfinance.Ticker object <FMB>
completed loading
completed loading
Errror with FMER, yfinance.Ticker object <FMER>
Errror with FMI, yfinance.Ticker object <FMI>
completed loading
Errror with FNBC, yfinance.Ticker object <FNBC>
Errror with FNFG, yfinance.Ticker object <FNFG>
Errror with FNGN, yfinance.Ticker object <FNGN>
completed loading
Errror with FNJN, yfinance.Ticke

completed loading
completed loading
completed loading
Errror with HEOP, yfinance.Ticker object <HEOP>
Errror with HERO, yfinance.Ticker object <HERO>
Errror with HFBC, yfinance.Ticker object <HFBC>
completed loading
Errror with HFFC, yfinance.Ticker object <HFFC>
completed loading
completed loading
completed loading
completed loading
completed loading
Errror with HIIQ, yfinance.Ticker object <HIIQ>
Errror with HILL, yfinance.Ticker object <HILL>
completed loading
Errror with HKTV, yfinance.Ticker object <HKTV>
completed loading
Errror with HLSS, yfinance.Ticker object <HLSS>
completed loading
Errror with HMIN, yfinance.Ticker object <HMIN>
completed loading


In [1]:
for SYMBOL in TICKERS: 
    try:
        derp = dataframe_collection[SYMBOL]

        plt.plot(derp.CumNetShortPercent)
        plt.legend(TICKERS)
    except:
        pass

NameError: name 'TICKERS' is not defined

In [ ]:
import pickle

basedir = '/home/pookie/data/derp/'

filename = 'dataframe_collection.pkl'
df_file = open(basedir+filename, 'wb')
pickle.dump(dataframe_collection, df_file)
df_file.close()

filename = 'shares_collection.pkl'
sc_file = open(basedir+filename, 'wb')
pickle.dump(shares_collection, sc_file)
sc_file.close()

In [ ]:
filename = 'dataframe_collection.pkl'
a_file = open(basedir+filename , "rb")

xxx  = pickle.load(a_file)

print(xxx)
